# SID Genetics Study Step 6: Genomic Analysis

## Objective
The purpose of this notebook is to run analyses on genetic data as part of a study on genetic predictors of statin-induced diabetes. These analyses include:

**Candidate variant study**: takes variants that have known associations with statin on-target LDL-C lowering and finds their associations with new-onset diabetes
- Variants tested: HMGCR (rs17238484, rs12916), LPA (rs10455872), SLC01B1 (rs4149056), APOE2 (rs7412), and APOE4 (rs429358)
- Statistical methods: stratified + adjusted Cox regression model, interaction test (Cochran's Q)
- Subsets tested: intention-to-treat, ≥30% decrease in LDL-C, per protocol, self-identified white

**Mini polygenic score (PS)**: takes the six candidate variants and combines their effect sizes into a "mini" PS for each participant
- Statistical methods: stratified + adjusted Cox regression model, interaction test (Cochran's Q)
- Subset tested: intention-to-treat

**Genome-wide Association Study (GWAS)**: uses All of Us' microarray data to find variants with genome-wide significant associations with statin induced diabetes
- Statistical methods: stratified + adjusted Cox regression model (using Regenie 4.1's time-to-event model), interaction test (Cochran's Q)
- Subsets tested: intention-to-treat, ≥30% decrease in LDL-C, per protocol, self-identified white

# Pull in Data

**Objective**: To load necessary packages + their citations and pull in data frames generated in previous notebooks necessary for genetic analysis.

In [ ]:
# install.packages('allofus')
# install.packages('tidyverse')
# install.packages('stats')
# install.packages('survival')
# install.packages('survminer')
# install.packages('qqman')
# install.packages('tableone')
# install.packages("cowplot")
# install.packages("tableone")
# install.packages('gridExtra')

library(allofus)
library(tidyverse)
library(stats)
library(survival)
library(survminer)
library(qqman)
library(tableone)
library(ggplot2)
library(cowplot)
library(gridExtra)

my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

In [ ]:
# Package citations
citation("allofus")
citation("tidyverse")
citation("stats")
citation("survival")
citation("survminer")
citation("qqman")
citation("tableone")
citation("ggplot2")
citation("cowplot")
citation("gridExtra")

In [ ]:
# Pull in data frames for each subset
system(paste0("gsutil cp ", my_bucket, "/sid_pheno_files/", "itt_df_v2.csv", " ."), intern=T)
itt_df <- read.csv("itt_df_v2.csv")

dim(itt_df)
# head(itt_df)

system(paste0("gsutil cp ", my_bucket, "/sid_pheno_files/", "ldl30_df_v2.csv", " ."), intern=T)
ldl30_df <- read.csv("ldl30_df_v2.csv")

dim(ldl30_df)
# head(ldl30_df)

system(paste0("gsutil cp ", my_bucket, "/sid_pheno_files/", "ldl_df_v3.csv", " ."), intern=T)
ldl30_df2 <- read.csv("ldl_df_v3.csv")

dim(ldl30_df2)
# head(ldl30_df2)

system(paste0("gsutil cp ", my_bucket, "/sid_pheno_files/", "per_protocol_df_v2.csv", " ."), intern=T)
per_protocol_df <- read.csv("per_protocol_df_v2.csv")

dim(per_protocol_df)
# head(per_protocol_df)

system(paste0("gsutil cp ", my_bucket, "/sid_pheno_files/", "white_df_v2.csv", " ."), intern=T)
white_df <- read.csv("white_df_v2.csv")

dim(white_df)
# head(white_df)

# LDL-C Analysis data frame
system(paste0("gsutil cp ", my_bucket, "/sid_pheno_files/", "ldl_df_v3.csv", " ."), intern=T)
ldl_df <- read.csv("ldl_df_v3.csv")

dim(ldl_df)
# head(ldl_df)

In [ ]:
# Pull in candidate variant data frames
system(paste0("gsutil cp ", my_bucket, "/sid_geno_files/candidate/", "sid_targets_chr5.raw", " ."), intern=T)
sid_targets_chr5 <- read_table("sid_targets_chr5.raw") %>% select(-c(PAT, MAT, SEX, PHENOTYPE))

dim(sid_targets_chr5)
# head(sid_targets_chr5)

system(paste0("gsutil cp ", my_bucket, "/sid_geno_files/candidate/", "sid_targets_chr6.raw", " ."), intern=T)
sid_targets_chr6 <- read_table("sid_targets_chr6.raw") %>% select(-c(PAT, MAT, SEX, PHENOTYPE))

dim(sid_targets_chr6)
# head(sid_targets_chr6)

system(paste0("gsutil cp ", my_bucket, "/sid_geno_files/candidate/", "sid_targets_chr12.raw", " ."), intern=T)
sid_targets_chr12 <- read_table("sid_targets_chr12.raw") %>% select(-c(PAT, MAT, SEX, PHENOTYPE))

dim(sid_targets_chr12)
# head(sid_targets_chr12)

system(paste0("gsutil cp ", my_bucket, "/sid_geno_files/candidate/", "sid_targets_chr19.raw", " ."), intern=T)
sid_targets_chr19 <- read_table("sid_targets_chr19.raw") %>% select(-c(PAT, MAT, SEX, PHENOTYPE))

dim(sid_targets_chr19)
# head(sid_targets_chr19)

# Merge candidate variant data frames
sid_targets <- full_join(sid_targets_chr5, sid_targets_chr6) %>%
                full_join(sid_targets_chr12) %>%
                full_join(sid_targets_chr19) %>%
                rename(rs17238484 = "chr5:75352671:G:T_G",
                       rs12916 = "chr5:75360714:T:C_T",
                       rs10455872 = "chr6:160589086:A:G_A",
                       rs4149056 = "chr12:21178615:T:C_T",
                       rs7412 = "chr19:44908822:C:T_C",
                       rs429358 = "chr19:44908684:T:C_T") %>%
                mutate(rs7412_swap = case_when(rs7412 == 2 ~ 0,
                                               rs7412 == 1 ~ 1,
                                               rs7412 == 0 ~ 2),
                      rs10455872_swap = case_when(rs10455872 == 2 ~ 0,
                                               rs10455872 == 1 ~ 1,
                                               rs10455872 == 0 ~ 2),
                      rs429358_swap = case_when(rs429358 == 2 ~ 0,
                                               rs429358 == 1 ~ 1,
                                               rs429358 == 0 ~ 2))

dim(sid_targets)
head(sid_targets)

In [ ]:
# Pull in GWAS results - itt
system(paste0("gsutil cp ", my_bucket, "/sid_geno_files/arrays/itt/SID_GWAS_array_statin_itt_MAF1_step2_array_time.regenie", " ."), intern=T)
array_statin_itt_MAF1 <- read_table("SID_GWAS_array_statin_itt_MAF1_step2_array_time.regenie")

dim(array_statin_itt_MAF1)
# head(array_statin_itt_MAF1)

system(paste0("gsutil cp ", my_bucket, "/sid_geno_files/arrays/itt/SID_GWAS_array_nu_itt_MAF1_step2_array_time.regenie", " ."), intern=T)
array_nu_itt_MAF1 <- read_table("SID_GWAS_array_nu_itt_MAF1_step2_array_time.regenie")

dim(array_nu_itt_MAF1)
# head(array_nu_itt_MAF1)

In [ ]:
# Pull in GWAS results - ldl30
system(paste0("gsutil cp ", my_bucket, "/sid_geno_files/arrays/ldl30/SID_GWAS_array_statin_ldl30_MAF25_step2_array_time.regenie", " ."), intern=T)
array_statin_ldl30_MAF25 <- read_table("SID_GWAS_array_statin_ldl30_MAF25_step2_array_time.regenie")

dim(array_statin_ldl30_MAF25)
# head(array_statin_ldl30_MAF25)

system(paste0("gsutil cp ", my_bucket, "/sid_geno_files/arrays/ldl30/SID_GWAS_array_nu_ldl30_MAF25_step2_array_time.regenie", " ."), intern=T)
array_nu_ldl30_MAF25 <- read_table("SID_GWAS_array_nu_ldl30_MAF25_step2_array_time.regenie")

dim(array_nu_ldl30_MAF25)
# head(array_nu_ldl30_MAF25)

In [ ]:
# Pull in GWAS results - per protocol
system(paste0("gsutil cp ", my_bucket, "/sid_geno_files/arrays/per_protocol/SID_GWAS_array_statin_per_protocol_MAF20_step2_array_time.regenie", " ."), intern=T)
array_statin_pp_MAF20 <- read_table("SID_GWAS_array_statin_per_protocol_MAF20_step2_array_time.regenie")

dim(array_statin_pp_MAF20)
# head(array_statin_pp_MAF20)

system(paste0("gsutil cp ", my_bucket, "/sid_geno_files/arrays/per_protocol/SID_GWAS_array_nu_per_protocol_MAF20_step2_array_time.regenie", " ."), intern=T)
array_nu_pp_MAF20 <- read_table("SID_GWAS_array_nu_per_protocol_MAF20_step2_array_time.regenie")

dim(array_nu_pp_MAF20)
# head(array_nu_pp_MAF20)

In [ ]:
# Pull in GWAS results - self-identified white
system(paste0("gsutil cp ", my_bucket, "/sid_geno_files/arrays/white/SID_GWAS_array_statin_white_MAF1_step2_array_time.regenie", " ."), intern=T)
array_statin_white_MAF1 <- read_table("SID_GWAS_array_statin_white_MAF1_step2_array_time.regenie")

dim(array_statin_white_MAF1)
# head(array_statin_white_MAF1)

system(paste0("gsutil cp ", my_bucket, "/sid_geno_files/arrays/white/SID_GWAS_array_nu_white_MAF1_step2_array_time.regenie", " ."), intern=T)
array_nu_white_MAF1 <- read_table("SID_GWAS_array_nu_white_MAF1_step2_array_time.regenie")

dim(array_nu_white_MAF1)
# head(array_nu_white_MAF1)

# Helper Functions - Candidate + mini PS

**Objective**: Create functions to streamline analysis of candidate variants and mini PS.

In [ ]:
# Function to calculate allele frequencies for each candidate variant
allele_frequency <- function(df, snp_start, snp_end) {
    
    # Function to count categories and NAs for a single column
    count_alleles <- function(column) {
      counts <- table(factor(column, levels = c(0, 1, 2), exclude = NULL), useNA = "always")
      counts <- as.data.frame(counts)
      names(counts) <- c("allele", "count")
      counts$allele <- as.character(counts$allele)
      counts$allele[is.na(counts$allele)] <- "NA"
      return(counts)
    }

    # Apply the function to each column and combine results
    allele_counts <- df %>%
                        summarize(across(c(snp_start:snp_end), ~ list(count_alleles(.x)))) %>%
                        pivot_longer(cols = everything(), names_to = "allele_name", values_to = "counts") %>%
                        unnest(counts)

    # Reshape the data frame for better readability
    allele_counts <- allele_counts %>%
                          pivot_wider(names_from = allele, values_from = count, values_fill = 0) %>%
                          rename(RR = "0",
                                 RA = "1",
                                 AA = "2")
                            
    
    # Find the allele frequencies for each allele
    allele_counts <- allele_counts %>%
                          mutate(ref_freq = (2*(RR)+RA)/(2*(RR+RA+AA)),
                                 alt_freq = (2*(AA)+RA)/(2*(RR+RA+AA)))
                        
    
    return(allele_counts)
    
}

In [ ]:
# Function to determine HR in each population for numeric variables (dosage or PS), split by treatment group
var_association <- function(df, var) {
    
    # Initialize the outputed data frame
    var_df <- data.frame(
                group = character(),
                variable = character(),
                beta = numeric(),
                HR = numeric(),
                lower_ci_HR = numeric(),
                upper_ci_HR = numeric(),
                HR_SE = numeric(),
                HR_p = numeric()      
            )
    
    # Split input data frame by treatment group
    grp_split <- split(
        df,
        df[,"group"],
        drop = FALSE
        )
    
    # Loop through split treatment group data frames
    for (i in names(grp_split)) {
        
        # If running this analysis in self-identified white subset, exclude population covariate,
        # else, include population covariate
        if(all(df$population == "White")) {
            
            int_formula <- as.formula(paste0("Surv(time, status) ~ ", var, " + index_age + sex_at_birth +
                                    low_hdl + high_tg + high_bmi + smoking_status + htn_status + gd_status +
                                    PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + 
                                    PC13 + PC14 + PC15 + PC16"))
            
        } else {
            
            int_formula <- as.formula(paste0("Surv(time, status) ~ ", var, " + index_age + population + sex_at_birth +
                                    low_hdl + high_tg + high_bmi + smoking_status + htn_status + gd_status +
                                    PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + 
                                    PC13 + PC14 + PC15 + PC16"))
            
        }
        
        # Run model
        int_m <- coxph(int_formula, data = grp_split[[i]])
        
        # Extract necessary outcomes from model output
        grp <- i
        var_name <- var
        grp_beta <- coef(summary(int_m))[paste0(var), "coef"]
        grp_HR <- coef(summary(int_m))[paste0(var), "exp(coef)"]
        grp_lower <- exp(confint(int_m))[paste0(var), "2.5 %"]
        grp_upper <- exp(confint(int_m))[paste0(var), "97.5 %"]
        grp_SE <- coef(summary(int_m))[paste0(var), "se(coef)"]
        grp_p <- coef(summary(int_m))[paste0(var), "Pr(>|z|)"]
        
        # Bind results to initialized data frame
        var_df <- rbind(var_df, data.frame(
                group = grp,
                variable = var_name,
                beta = grp_beta,
                HR = grp_HR,
                lower_ci_HR = grp_lower,
                upper_ci_HR = grp_upper,
                HR_SE = grp_SE,
                HR_p = grp_p))
        
    }
    
    return(var_df)
    
}

In [ ]:
# Put variants together into one data frame
candidate_results <- function(df, snp_start, snp_end) {
    
    # Get index of first SNP and last
    istart <- which(names(df) == snp_start)
    iend <- which(names(df) == snp_end)
    
    # Initialize data frame to hold results for all variants
    snp_df <- data.frame(
                group = character(),
                variable = character(),
                beta = numeric(),
                HR = numeric(),
                lower_ci_HR = numeric(),
                upper_ci_HR = numeric(),
                HR_SE = numeric(),
                HR_p = numeric()      
            )
    
    # Bind data frames with results for each variant
    for (snp in colnames(df[, istart:iend])) {
        snp_df <- rbind(snp_df, var_association(df, snp))
    }
    
    return(snp_df)
    
}


In [ ]:
# Function to calculate P of interaction

p_int_calculator <- function(t_beta, t_se, c_beta, c_se) {
    # Sample data
    beta <- c(t_beta, c_beta)  # beta coefficients for each stratum
    se <- c(t_se, c_se)   # standard errors for each stratum

    # Calculate the weights for each stratum
    weights <- 1 / (se^2)

    # Calculate the weighted mean effect size
    weighted_mean <- sum(weights * beta) / sum(weights)

    # Calculate Cochran's Q
    Q <- sum(weights * (beta - weighted_mean)^2)

    # Degrees of freedom
    df <- length(beta) - 1

    # Calculate the p-value for Cochran's Q test
    p_value <- pchisq(Q, df, lower.tail = FALSE)

    # Print the results
    return(p_value = p_value)
}

In [ ]:
# Function to create a data frame with all interaction test results
int_test <- function(df, snp_df, snp_start, snp_end) {
    
    # Get index of first SNP and last
    istart <- which(names(df) == snp_start)
    iend <- which(names(df) == snp_end)
    
    # Initialize data frame to hold results
    int_p_df <- data.frame(snp = character(), p_int_calc = numeric())
    
    # Loop through each SNP and generate P of interaction using calculator function
    for (snp in colnames(df[, istart:iend])) {
        
        # Calculate P of interaction
        calc_p <- p_int_calculator(snp_df[snp_df$variable == snp & snp_df$group == "non-user", "beta"], 
                                  snp_df[snp_df$variable == snp & snp_df$group == "non-user", "HR_SE"],
                                  snp_df[snp_df$variable == snp & snp_df$group == "user", "beta"], 
                                  snp_df[snp_df$variable == snp & snp_df$group == "user", "HR_SE"])
        
        # Get variant name
        snp_name <- snp
        # Get P of interaction
        grp_p_calc <- calc_p
        
        # Bind results to data frame
        int_p_df <- rbind(int_p_df, data.frame(
                            snp = snp_name,
                            p_int_calc = grp_p_calc))
        }
    
    return(int_p_df)
    
}

In [ ]:
# Function to determine HR in each PS quintile, split by treatment group
psq_association <- function(df, quint, quintiles_vector) {
    
    # Initialize quintile data frame
    quint_df <- data.frame(
                group = character(),
                quintile = character(),
                HR = numeric(),
                beta = numeric(),
                lower_ci_HR = numeric(),
                upper_ci_HR = numeric(),
                HR_SE = numeric(),
                HR_p = numeric()      
            )
    
    # Split cohort by treatment group
    grp_split <- split(
        df,
        df[,"group"],
        drop = FALSE
        )
    
    # Loop through both treatment groups
    for (i in names(grp_split)) {
        
        # If running this analysis in self-identified white subset, exclude population covariate,
        # else, include population covariate
        if(all(df$population == "White")) {
            
            int_formula <- as.formula(paste0("Surv(time, status) ~ ", quint, " + index_age + sex_at_birth +
                                    low_hdl + high_tg + high_bmi + smoking_status + htn_status + gd_status +
                                    PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + 
                                    PC13 + PC14 + PC15 + PC16"))
            
        } else {
            
            int_formula <- as.formula(paste0("Surv(time, status) ~ ", quint, " + index_age + population + sex_at_birth +
                                    low_hdl + high_tg + high_bmi + smoking_status + htn_status + gd_status +
                                    PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + 
                                    PC13 + PC14 + PC15 + PC16"))
            
        }
        
        # Run the interaction model
        int_m <- coxph(int_formula, data = grp_split[[i]])
        
        # Extract necessary outputs from model summary for the second quintile
        grp <- i
        quint_name <- quintiles_vector[1]
        grp_beta <- coef(summary(int_m))[quintiles_vector[1], "coef"]
        grp_HR <- coef(summary(int_m))[quintiles_vector[1], "exp(coef)"]
        grp_lower <- exp(confint(int_m))[quintiles_vector[1], "2.5 %"]
        grp_upper <- exp(confint(int_m))[quintiles_vector[1], "97.5 %"]
        grp_SE <- coef(summary(int_m))[quintiles_vector[1], "se(coef)"]
        grp_p <- coef(summary(int_m))[quintiles_vector[1], "Pr(>|z|)"]
        
        # Bind outputs to initialize data frame 
        quint_df <- rbind(quint_df, data.frame(
                group = grp,
                quintile = quint_name,
                HR = grp_HR,
                beta = grp_beta,
                lower_ci_HR = grp_lower,
                upper_ci_HR = grp_upper,
                HR_SE = grp_SE,
                HR_p = grp_p))
        
        # Loop through the rest of the quintiles and bind extracted outputs to quintile data frame
        for (q in quintiles_vector[seq(2,4)]) {
            
            grp <- i
            quint_name <- q
            grp_beta <- coef(summary(int_m))[q, "coef"]
            grp_HR <- coef(summary(int_m))[q, "exp(coef)"]
            grp_lower <- exp(confint(int_m))[q, "2.5 %"]
            grp_upper <- exp(confint(int_m))[q, "97.5 %"]
            grp_SE <- coef(summary(int_m))[q, "se(coef)"]
            grp_p <- coef(summary(int_m))[q, "Pr(>|z|)"]
        
            quint_df <- rbind(quint_df, data.frame(
                    group = grp,
                    quintile = q,
                    HR = grp_HR,
                    beta = grp_beta,
                    lower_ci_HR = grp_lower,
                    upper_ci_HR = grp_upper,
                    HR_SE = grp_SE,
                    HR_p = grp_p))
            
        }
        
        
    }
    
    return(quint_df)
    
}

# Candidate Studies

**Objective**: The purpose of this section is to model the association between NOD and six candidate variants that have demonstrated an effect on statin on-target LDL-C lowering. To do this, we first generate the hazard ratios (HR) for each variant in statin users and non-users separately, which gives us an estimate of the effect of the variant in each treatment group. Then, we use Cochran's Q test to find the heterogenity between the effect of the variant in statin users vs. non-users. If the P value for a variant is below a Boneferroni adjusted 0.008333 (adjusted for multiple comparisons for six variants), it is considered to have a significant interaction with NOD, and thus point to an association with statin-induced diabetes. A P-value of less than 0.05 is considered suggestive of a significant interaction.

## Intention-to-treat

In [ ]:
# Prep data frame for itt subset
itt_snp_df <- left_join(itt_df, sid_targets)

dim(itt_snp_df)
# head(itt_snp_df)

In [ ]:
# Calculate allele frequencies overall
allele_frequency(itt_snp_df, "rs17238484", "rs429358_swap")

# Calculate allele frequencies by NOD status (1 = NOD, 0 = no NOD)
allele_frequency(itt_snp_df %>% filter(status == 1), "rs17238484", "rs429358_swap")
allele_frequency(itt_snp_df %>% filter(status == 0), "rs17238484", "rs429358_swap")

In [ ]:
# Calculate HRs for each SNP in each treatment group
itt_int_snps <- candidate_results(itt_snp_df, "rs17238484", "rs429358_swap")
itt_int_snps

# Calculate the p-interaction with statins for each SNP
int_test(itt_snp_df, itt_int_snps, "rs17238484", "rs429358_swap")

## ≥30% Decrease in LDL-C

In [ ]:
# Prep data frame for ≥30% decrease in LDL-C subset
ldl30_snp_df <- left_join(ldl30_df, sid_targets)

dim(ldl30_snp_df)
# head(ldl30_snp_df)

In [ ]:
# Calculate allele frequencies overall
allele_frequency(ldl30_snp_df, "rs17238484", "rs429358_swap")

# Calculate allele frequencies by NOD status (1 = NOD, 0 = no NOD)
allele_frequency(ldl30_snp_df %>% filter(status == 1), "rs17238484", "rs429358_swap")
allele_frequency(ldl30_snp_df %>% filter(status == 0), "rs17238484", "rs429358_swap")

In [ ]:
# Calculate HRs for each SNP in each treatment group
ldl30_int_snps <- candidate_results(ldl30_snp_df, "rs17238484", "rs429358_swap")
ldl30_int_snps

# Calculate the p-interaction with statins for each SNP
int_test(ldl30_snp_df, ldl30_int_snps, "rs17238484", "rs429358_swap")

## Per Protocol

In [ ]:
# Prep data frame for per protocol subset
pp_snp_df <- left_join(per_protocol_df, sid_targets)

dim(pp_snp_df)
# head(pp_snp_df)

In [ ]:
# Calculate allele frequencies overall
allele_frequency(pp_snp_df, "rs17238484", "rs429358_swap")

# Calculate allele frequencies by NOD status (1 = NOD, 0 = no NOD)
allele_frequency(pp_snp_df %>% filter(status == 1), "rs17238484", "rs429358_swap")
allele_frequency(pp_snp_df %>% filter(status == 0), "rs17238484", "rs429358_swap")

In [ ]:
# Calculate HRs for each SNP in each treatment group
pp_int_snps <- candidate_results(pp_snp_df, "rs17238484", "rs429358_swap")
pp_int_snps

# Calculate the p-interaction with statins for each SNP
int_test(pp_snp_df, pp_int_snps, "rs17238484", "rs429358_swap")

## Self-identified White

In [ ]:
# Prep data frame for self-identified white subset
white_snp_df <- left_join(white_df, sid_targets)

dim(white_snp_df)
# head(white_snp_df)

In [ ]:
# Calculate allele frequencies overall
allele_frequency(white_snp_df, "rs17238484", "rs429358_swap")

# Calculate allele frequencies by NOD status (1 = NOD, 0 = no NOD)
allele_frequency(white_snp_df %>% filter(status == 1), "rs17238484", "rs429358_swap")
allele_frequency(white_snp_df %>% filter(status == 0), "rs17238484", "rs429358_swap")

In [ ]:
# Calculate HRs for each SNP in each treatment group
white_int_snps <- candidate_results(white_snp_df, "rs17238484", "rs429358_swap")
white_int_snps

# Calculate the p-interaction with statins for each SNP
int_test(white_snp_df, white_int_snps, "rs17238484", "rs429358_swap")

# Mini PS

**Objective**: The purpose of this step is to determine the association of all the candidate variants with SID at once. Since the individual effects of the candidate variants are so small, despite our fairly large sample size, we do not have enough power to detect these associations, which likely exist since all candidate variant interactions followed the expected statin on-target direction of effect. To remedy this, we want to add all the effects together into a polygenic score (PS) and find the association with SID, thus providing stronger evidence that on-target variants are associated with an increased risk of SID. 

In [ ]:
# Clean up the candidate variant data frame for mini PS
mini_ps_snps <- sid_targets %>% 
                    select(-c(rs7412,
                              rs10455872,
                              rs429358)) %>% rename(rs7412 = rs7412_swap,
                                               rs10455872 = rs10455872_swap,
                                               rs429358 = rs429358_swap) %>% # Prepare the correct version of rs7412
                    right_join(itt_df) # Get all covariate data

dim(mini_ps_snps)
head(mini_ps_snps)

In [ ]:
# Generating a random 50/50 split in the intention-to-treat cohort
# First 50% will be used to generate weights
# Other 50% will be used to model association between PS and SID

# Create df with statin users only
mini_ps_statin <- mini_ps_snps %>% filter(group == 'user')

# 50% of the sample size
sample_size <- floor(0.50 * nrow(mini_ps_statin))

# Set seed to make partition reproducible
set.seed(42)

# Generate sample indicies
itrain <- sample(seq_len(nrow(mini_ps_statin)), size = sample_size)

# Select person IDs for sampled statin users
mini_ps_snps_train_ids <- mini_ps_statin[itrain, ] %>% select(person_id)
mini_ps_snps_test_ids <- mini_ps_statin[-itrain, ] %>% select(person_id)

# Filter for match groups to maintain matching structures
mini_ps_snps_train <- mini_ps_snps %>% filter(match_group %in% mini_ps_snps_train_ids$person_id)
mini_ps_snps_test <- mini_ps_snps %>% filter(match_group %in% mini_ps_snps_test_ids$person_id)

# Check
dim(mini_ps_snps_train) 
# head(mini_ps_snps_train)

dim(mini_ps_snps_test)
# head(mini_ps_snps_test)

In [ ]:
# Generate PS weights

# Set start and end rs IDs for indexing
istart <- which(names(mini_ps_snps_train) == "rs17238484")
iend <- which(names(mini_ps_snps_train) == "rs429358")

# Initialize PS data frame
ps_snp_df <- data.frame(
    
                group = character(),
                snp = character(),
                HR = numeric(),
                lower_ci_HR = numeric(),
                upper_ci_HR = numeric(),
                HR_SE = numeric(),
                HR_p = numeric()      
            )

# Run helper function to generate weights in non-users and statin users on all candidate variants
for (snp in colnames(mini_ps_snps_train[, istart:iend])) {
        ps_snp_df <- rbind(ps_snp_df, var_association(mini_ps_snps_train, snp))
    }

# View stratified HRs
ps_snp_df

# Calculate HR ratios (estimated interaction effect)
ps_snp_df <- ps_snp_df %>% 
                select(group, variable, HR) %>%
                spread(key = group, value = HR) %>%
                mutate(HR_ratio = user / `non-user`)# %>%
                # select(snp, HR_ratio)

ps_snp_df

In [ ]:
# Calculate PS

# Reshape dosages data frame from wide to long format
mini_ps_snps_test_long <- mini_ps_snps_test %>% 
    select(FID, IID, group, rs17238484, rs12916, rs10455872, rs4149056, rs429358, rs7412) %>%
    pivot_longer(cols = starts_with("rs"), names_to = "variable", values_to = "dosage")

# Merge effect sizes with dosages based on group and SNP
merged_data <- merge(mini_ps_snps_test_long, ps_snp_df, by = c("variable"))

# Calculate the PS for each individual
ps_scores <- merged_data %>%
  group_by(IID) %>%
  summarize(PS = sum(dosage * HR_ratio)) %>%
  mutate(PS_norm = as.numeric(scale(PS))) # Normalize PS for more interpretible continuous associations

# Print PS scores
dim(ps_scores)
# head(ps_scores)

In [ ]:
# Join PS with covariate data and split into quintiles
mini_ps_snps_df <- full_join(mini_ps_snps_test, ps_scores) %>% 
                        mutate(quintile = ntile(PS, 5),
                               quintile = paste0("Q", quintile),
                               quintile_norm = ntile(PS_norm, 5),
                               quintile_norm = paste0("Q", quintile_norm))

In [ ]:
# Summarize untransformed PS

# Summarize PS by treatment group
mini_ps_snps_df %>% group_by(group) %>%
            summarize(percent_t2d = round(sum(t2d_status == "Event")/n(), digits = 3)*100,
                      min = min(PS, na.rm = TRUE),
                      median = median(PS, na.rm = TRUE),
                      max = max(PS, na.rm = TRUE))

# Define variables for Table 1
myVars <- c("t2d_status", "PS")
catVars <- c("t2d_status")

# Create Table 1 to see differences in PS - by group
tab_ps_scores <- CreateTableOne(vars = myVars, strata = "group", data = mini_ps_snps_df, factorVars = catVars, test = TRUE)
as.data.frame(print(tab_ps_scores, noSpaces = TRUE, printToggle = FALSE))

# Create Table 1 to see differences in PS - by T2D status
tab_ps_scores <- CreateTableOne(vars = myVars, strata = "t2d_status", data = mini_ps_snps_df, factorVars = catVars, test = TRUE)
as.data.frame(print(tab_ps_scores, noSpaces = TRUE, printToggle = FALSE))

# Create grouped boxplots
boxplot <- ggplot(mini_ps_snps_df, aes(x = group, y = PS, fill = group)) +
                geom_boxplot() +
                coord_flip() +
                theme_void() +
                scale_fill_manual(values = c("#C32882", "#178CCB")) +
                theme(legend.position = "none")

# Create grouped density plots
density_plot <- ggplot(mini_ps_snps_df, aes(x = PS, fill = group)) +
                    geom_density(alpha = 0.5) +
                    labs(x = "PS Scores", y = "Density") +
                    scale_fill_manual(values = c("#C32882", "#178CCB")) +
                    theme_minimal()

# Combine plots
combined_plot <- plot_grid(boxplot, density_plot, ncol = 1, rel_heights = c(1, 8))
print(combined_plot)

In [ ]:
# Summarize normalized PS

# Summarize PS by treatment group
mini_ps_snps_df %>% group_by(group) %>%
            summarize(percent_t2d = round(sum(t2d_status == "Event")/n(), digits = 3)*100,
                      min = min(PS_norm, na.rm = TRUE),
                      median = median(PS_norm, na.rm = TRUE),
                      max = max(PS_norm, na.rm = TRUE))

# Define variables for Table 1
myVars <- c("t2d_status", "PS_norm")
catVars <- c("t2d_status")

# Create Table 1 to see differences in PS - by group
tab_ps_scores <- CreateTableOne(vars = myVars, strata = "group", data = mini_ps_snps_df, factorVars = catVars, test = TRUE)
as.data.frame(print(tab_ps_scores, noSpaces = TRUE, printToggle = FALSE))

# Create Table 1 to see differences in PS - by T2D status
tab_ps_scores <- CreateTableOne(vars = myVars, strata = "t2d_status", data = mini_ps_snps_df, factorVars = catVars, test = TRUE)
as.data.frame(print(tab_ps_scores, noSpaces = TRUE, printToggle = FALSE))

# Create grouped boxplots
boxplot <- ggplot(mini_ps_snps_df, aes(x = group, y = PS_norm, fill = group)) +
                geom_boxplot() +
                coord_flip() +
                theme_void() +
                scale_fill_manual(values = c("#C32882", "#178CCB")) +
                theme(legend.position = "none")

# Create grouped density plots
density_plot <- ggplot(mini_ps_snps_df, aes(x = PS_norm, fill = group)) +
                    geom_density(alpha = 0.5) +
                    labs(x = "Normalized PS Scores", y = "Density") +
                    scale_fill_manual(values = c("#C32882", "#178CCB")) +
                    theme_minimal()

# Combine plots
combined_plot <- plot_grid(boxplot, density_plot, ncol = 1, rel_heights = c(1, 8))
print(combined_plot)

In [ ]:
# Check mini PS data frame
dim(mini_ps_snps_df)
# head(mini_ps_snps_df)

In [ ]:
# Summarize mini PS by quintiles - untransformed
mini_ps_snps_df %>% group_by(quintile) %>% summarize(count = n(),
                                                      non_users = sum(group == "non-user"),
                                                      users = sum(group == "user"),
                                                      percent_t2d = round(sum(t2d_status == "Event")/n(), digits = 3)*100,
                                                      min = min(PS),
                                                      median = median(PS),
                                                      mean = mean(PS),
                                                      max = max(PS))

In [ ]:
# Summarize mini PS by quintiles - normalized
mini_ps_snps_df %>% group_by(quintile_norm) %>% summarize(count = n(),
                                                      non_users = sum(group == "non-user"),
                                                      users = sum(group == "user"),
                                                      percent_t2d = round(sum(t2d_status == "Event")/n(), digits = 3)*100,
                                                      min = min(PS_norm),
                                                      median = median(PS_norm),
                                                      mean = mean(PS_norm),
                                                      max = max(PS_norm))

### PS Associations

#### Untransformed PS

In [ ]:
# Association of NOD and a 1-point increase in PS, stratified by treatment group
ps_overall <- var_association(mini_ps_snps_df, "PS")
ps_overall

In [ ]:
# Heterogeneity test for interaction between PS and NOD
int_test(mini_ps_snps_df, ps_overall, "PS", "PS")

In [ ]:
# Calculate HRs for each SNP in each treatment group
ps_quintiles <- psq_association(mini_ps_snps_df, 'quintile', 
                                  c("quintileQ2", "quintileQ3", "quintileQ4", "quintileQ5"))

# Calculate proportion of NOD cases for each treatment group and quintile
nod_prop <- mini_ps_snps_df %>% group_by(group, quintile) %>% 
                summarize(group = group, 
                          quintile = paste0('quintile', quintile), 
                          prop_nod = sum(t2d_status == "Event")/n()) %>%
                distinct(.keep_all = TRUE)

# Join data frames
ps_quintiles <- left_join(ps_quintiles, nod_prop)
ps_quintiles

# Initialize P of interaction data frame
int_p_df <- data.frame(q = character(), p_int_calc = numeric())

# Loop through each quintile and calculate P of interaction for each
for (q in c("quintileQ2", "quintileQ3", "quintileQ4", "quintileQ5")) {
        
        # Calculate P of interaction
        calc_p <- p_int_calculator(ps_quintiles[ps_quintiles$quintile == q & ps_quintiles$group == "non-user", "beta"], 
                                  ps_quintiles[ps_quintiles$quintile == q & ps_quintiles$group == "non-user", "HR_SE"],
                                  ps_quintiles[ps_quintiles$quintile == q & ps_quintiles$group == "user", "beta"], 
                                  ps_quintiles[ps_quintiles$quintile == q & ps_quintiles$group == "user", "HR_SE"])
        
        # Assign necessary variables
        q_name <- q
        grp_p_calc <- calc_p
        
        # Bind results together
        int_p_df <- rbind(int_p_df, data.frame(
                            quintile = q_name,
                            p_int_calc = grp_p_calc))
    }
    
int_p_df

#### Normalized PS

In [ ]:
# Association of NOD and a 1-point increase in PS, stratified by treatment group
ps_overall <- var_association(mini_ps_snps_df, "PS_norm")
ps_overall

In [ ]:
# Heterogeneity test for interaction between PS and NOD
int_test(mini_ps_snps_df, ps_overall, "PS_norm", "PS_norm")

In [ ]:
# Calculate HRs for each SNP in each treatment group
ps_quintiles <- psq_association(mini_ps_snps_df, 'quintile_norm', 
                                  c("quintile_normQ2", "quintile_normQ3", "quintile_normQ4", "quintile_normQ5"))

# Calculate proportion of NOD cases for each treatment group and quintile
nod_prop <- mini_ps_snps_df %>% group_by(group, quintile_norm) %>% 
                summarize(group = group, 
                          quintile = paste0('quintile_norm', quintile_norm), 
                          prop_nod = sum(t2d_status == "Event")/n()) %>%
                distinct(.keep_all = TRUE)

# Join data frames
ps_quintiles <- left_join(ps_quintiles, nod_prop)
ps_quintiles

# Initialize P of interaction data frame
int_p_df <- data.frame(q = character(), p_int_calc = numeric())

# Loop through each quintile and calculate P of interaction for each
for (q in c("quintile_normQ2", "quintile_normQ3", "quintile_normQ4", "quintile_normQ5")) {
        
        # Calculate P of interaction
        calc_p <- p_int_calculator(ps_quintiles[ps_quintiles$quintile == q & ps_quintiles$group == "non-user", "beta"], 
                                  ps_quintiles[ps_quintiles$quintile == q & ps_quintiles$group == "non-user", "HR_SE"],
                                  ps_quintiles[ps_quintiles$quintile == q & ps_quintiles$group == "user", "beta"], 
                                  ps_quintiles[ps_quintiles$quintile == q & ps_quintiles$group == "user", "HR_SE"])
        
        # Assign necessary variables
        q_name <- q
        grp_p_calc <- calc_p
        
        # Bind results together
        int_p_df <- rbind(int_p_df, data.frame(
                            quintile = q_name,
                            p_int_calc = grp_p_calc))
    }
    
int_p_df

### Create Forest Plots

In [ ]:
# Create a forest plot 

# Clean up quintile names before merging
ps_fp <- ps_quintiles %>%
  mutate(quintile = gsub("quintile_norm", "", quintile))  # Remove "quintile_norm" prefix

int_p_df <- int_p_df %>%
  mutate(quintile = gsub("quintile_norm", "", quintile))  # Remove "quintile_norm" prefix

# Merge the two dataframes
ps_fp <- ps_fp %>%
  left_join(int_p_df, by = c("quintile"))

# Create labels for the plot
ps_fp <- ps_fp %>%
  mutate(
    prop_nod_pct = prop_nod * 100, # Convert prop_nod to percentage
    label = sprintf("%.2f (%.2f-%.2f)", HR, lower_ci_HR, upper_ci_HR) # Create formatted string for display
  )

# Create the forest plot
p <- ggplot(ps_fp, aes(y = quintile)) +
  geom_vline(xintercept = 1, linetype = "dashed", color = "gray50") + # Add vertical line at HR = 1
  geom_point(aes(x = HR, shape = group, color = group), position = position_dodge(width = 0.5)) + # Add points and confidence intervals
  geom_errorbarh(aes(xmin = lower_ci_HR, xmax = upper_ci_HR, linetype = group, color = group), 
                 height = 0.2, position = position_dodge(width = 0.5)) +
  scale_x_continuous(breaks = seq(0.5, 2, 0.25)) + # Customize appearance
  scale_color_manual(values = c("blue", "red")) +
  scale_linetype_manual(values = c("dashed", "solid")) +
  theme_bw() +
  labs(x = "Hazard Ratio (95% CI)", y = "Quintile") +
  theme(legend.position = "top")

# Create a table with the numerical results and round all numeric columns to 2 decimal places
results_table <- ps_fp %>%
  select(quintile, group, HR, lower_ci_HR, upper_ci_HR, HR_p, prop_nod_pct, p_int_calc) %>%
  arrange(quintile, group) %>%
  mutate(across(where(is.numeric), ~round(., 3)))

# Print both plot and table
p
results_table

In [ ]:
# Function to determine HR in each population for numeric variables (dosage or PS), split by treatment group
var_association_ldl <- function(df, ps, var) {
    
    # Initialize the outputed data frame
    var_df <- data.frame(
                group = character(),
                variable = character(),
                beta = numeric(),
                HR = numeric(),
                lower_ci_HR = numeric(),
                upper_ci_HR = numeric(),
                HR_SE = numeric(),
                HR_p = numeric()      
            )
    
    # Split input data frame by treatment group
    grp_split <- split(
        df,
        df[,"group"],
        drop = FALSE
        )
    
    # Loop through split treatment group data frames
    for (i in names(grp_split)) {
        
        # If running this analysis in self-identified white subset, exclude population covariate,
        # else, include population covariate
        if(all(df$population == "White")) {
            
            int_formula <- as.formula(paste0(ps " ~ ", var, " + index_age + sex_at_birth +
                                    low_hdl + high_tg + high_bmi + smoking_status + htn_status + gd_status +
                                    PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + 
                                    PC13 + PC14 + PC15 + PC16"))
            
        } else {
            
            int_formula <- as.formula(paste0(ps " ~ ", var, " + index_age + population + sex_at_birth +
                                    low_hdl + high_tg + high_bmi + smoking_status + htn_status + gd_status +
                                    PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + 
                                    PC13 + PC14 + PC15 + PC16"))
            
        }
        
        # Run model
        int_m <- coxph(int_formula, data = grp_split[[i]])
        
        # Extract necessary outcomes from model output
        grp <- i
        var_name <- var
        grp_beta <- coef(summary(int_m))[paste0(var), "coef"]
        grp_HR <- coef(summary(int_m))[paste0(var), "exp(coef)"]
        grp_lower <- exp(confint(int_m))[paste0(var), "2.5 %"]
        grp_upper <- exp(confint(int_m))[paste0(var), "97.5 %"]
        grp_SE <- coef(summary(int_m))[paste0(var), "se(coef)"]
        grp_p <- coef(summary(int_m))[paste0(var), "Pr(>|z|)"]
        
        # Bind results to initialized data frame
        var_df <- rbind(var_df, data.frame(
                group = grp,
                variable = var_name,
                beta = grp_beta,
                HR = grp_HR,
                lower_ci_HR = grp_lower,
                upper_ci_HR = grp_upper,
                HR_SE = grp_SE,
                HR_p = grp_p))
        
    }
    
    return(var_df)
    
}

In [ ]:
# Association of NOD and a 1-point increase in PS, stratified by treatment group
mini_ps_snps_df_ldl <- left_join(mini_ps_snps_df, ldl_df)

dim(mini_ps_snps_df_ldl)
head(mini_ps_snps_df_ldl)

ps_overall <- var_association_ldl(mini_ps_snps_df_ldl, "PS_norm", "max_decrease_ldl")
ps_overall

# Helper Functions - GWAS

**Objective**: Create functions to streamline analysis of genome-wide association studies (GWAS).

In [ ]:
# Function to create a data frame with interaction results for GWAS
interaction_test <- function(statin_res, nu_res) {
    
    # Combine 
    int_results <- full_join(statin_res, nu_res, by = join_by(CHROM, GENPOS, ID, ALLELE0, ALLELE1)) %>%
                            group_by(ID) %>%
                            mutate(p_int = p_int_calculator(BETA.x, SE.x, BETA.y, SE.y)) %>%
                            arrange(p_int)
    
    return(int_results)
    
}

# GWAS - Array

**Objective**: The purpose of this section is to find out if any variants have a genome-wide association with SID. This is done by first running GWAS in statin users and non-users separately to find the effect size of each variant in their respective treatment groups. The next steps is to use the Cochran's Q test to find the heterogeneity between each treatment group for each variant. If the P value from this test is less than a Boneferroni corrected 5 x 10<sup>-8</sup>, the interaction is considered genome-wide significant.

## Intention-to-treat

In [ ]:
# Generate Manhattan plot + qq plot for statin users
options(repr.plot.width=15, repr.plot.height=8)
manhattan(array_statin_itt_MAF1, chr="CHROM", bp="GENPOS", snp="ID", p="LOG10P", annotatePval = 0.01)
qq(array_statin_itt_MAF1$LOG10P)

# Generate top 10 SNPs
head(array_statin_itt_MAF1 %>% arrange(LOG10P), 10)

In [ ]:
# Generate Manhattan plot + qq plot for non-users
manhattan(array_nu_itt_MAF1, chr="CHROM", bp="GENPOS", snp="ID", p="LOG10P", annotatePval = 0.01)
qq(array_nu_itt_MAF1$LOG10P)

# Generate top 10 SNPs
head(array_nu_itt_MAF1 %>% arrange(LOG10P), 10)

In [ ]:
# Generate interaction p-values
array_itt_MAF1_int <- interaction_test(array_statin_itt_MAF1, array_nu_itt_MAF1)

dim(array_itt_MAF1_int)
head(array_itt_MAF1_int)

In [ ]:
# Generate interaction Manhattan plot + qq plot
manhattan(array_itt_MAF1_int, chr="CHROM", bp="GENPOS", snp="ID", p="p_int", annotatePval = 0.01)
qq(array_itt_MAF1_int$p_int)

# Generate top 10 SNPs
head(array_itt_MAF1_int %>% arrange(p_int), 10)

## ≥30% Decrease in LDL-C

In [ ]:
# Generate Manhattan plot + qq plot for statin users
manhattan(array_statin_ldl30_MAF25, chr="CHROM", bp="GENPOS", snp="ID", p="LOG10P", annotatePval = 0.01)
qq(array_statin_ldl30_MAF25$LOG10P)

# Generate top 10 SNPs
head(array_statin_ldl30_MAF25 %>% arrange(LOG10P), 10)

In [ ]:
# Generate Manhattan plot + qq plot for non-users
manhattan(array_nu_ldl30_MAF25, chr="CHROM", bp="GENPOS", snp="ID", p="LOG10P", annotatePval = 0.01)
qq(array_nu_ldl30_MAF25$LOG10P)

# Generate top 10 SNPs
head(array_nu_ldl30_MAF25 %>% arrange(LOG10P), 10)

In [ ]:
# Generate interaction p-values
array_ldl30_MAF25_int <- interaction_test(array_statin_ldl30_MAF25, array_nu_ldl30_MAF25)

In [ ]:
# Generate interaction Manhattan plot + qq plot
manhattan(array_ldl30_MAF25_int, chr="CHROM", bp="GENPOS", snp="ID", p="p_int", annotatePval = 0.01)
qq(array_ldl30_MAF25_int$p_int)

# Generate top 10 SNPs
head(array_ldl30_MAF25_int %>% arrange(p_int), 10)

## Per Protocol

In [ ]:
# Generate Manhattan plot + qq plot for statin users
manhattan(array_statin_pp_MAF20, chr="CHROM", bp="GENPOS", snp="ID", p="LOG10P", annotatePval = 0.01)
qq(array_statin_pp_MAF20$LOG10P)

# Generate top 10 SNPs
head(array_statin_pp_MAF20 %>% arrange(LOG10P), 10)

In [ ]:
# Generate Manhattan plot + qq plot for non-users
manhattan(array_nu_pp_MAF20, chr="CHROM", bp="GENPOS", snp="ID", p="LOG10P", annotatePval = 0.01)
qq(array_nu_pp_MAF20$LOG10P)

# Generate top 10 SNPs
head(array_nu_pp_MAF20 %>% arrange(LOG10P), 10)

In [ ]:
# Generate interaction p-values
array_pp_MAF20_int <- interaction_test(array_statin_pp_MAF20, array_nu_pp_MAF20)

In [ ]:
# Generate interaction Manhattan plot + qq plot
manhattan(array_pp_MAF20_int, chr="CHROM", bp="GENPOS", snp="ID", p="p_int", annotatePval = 0.01)
qq(array_pp_MAF20_int$p_int)

# Generate top 10 SNPs
head(array_pp_MAF20_int %>% arrange(p_int), 10)

## White

In [ ]:
# Generate Manhattan plot + qq plot for statin users
manhattan(array_statin_white_MAF1, chr="CHROM", bp="GENPOS", snp="ID", p="LOG10P", annotatePval = 0.01)
qq(array_statin_white_MAF1$LOG10P)

# Generate tqaE34Rop 10 SNPs
head(array_statin_white_MAF1 %>% arrange(LOG10P), 10)

In [ ]:
# Generate Manhattan plot + qq plot for non-users
manhattan(array_nu_white_MAF1, chr="CHROM", bp="GENPOS", snp="ID", p="LOG10P", annotatePval = 0.01)
qq(array_nu_white_MAF1$LOG10P)

# Generate top 10 SNPs
head(array_nu_white_MAF1 %>% arrange(LOG10P), 10)

In [ ]:
# Generate interaction p-values
array_white_MAF1_int <- interaction_test(array_statin_white_MAF1, array_nu_white_MAF1)

In [ ]:
# Generate interaction Manhattan plot + qq plot
manhattan(array_white_MAF1_int, chr="CHROM", bp="GENPOS", snp="ID", p="p_int", annotatePval = 0.01)
qq(array_white_MAF1_int$p_int)

# Generate top 10 SNPs
head(array_white_MAF1_int %>% arrange(p_int), 10)